<a href="https://colab.research.google.com/github/rajeshradhakrishnanmvk/ML2025/blob/master/AudioTrans002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq --upgrade openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [3]:
from google.colab import files
_ = files.upload()

Saving Alloy_tts-1_1x_RobertFrost_WoodsAreLovely2024-07-19T05_26_54-264Z.mp3 to Alloy_tts-1_1x_RobertFrost_WoodsAreLovely2024-07-19T05_26_54-264Z.mp3


In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

In [5]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.environ['OPENAI_API_KEY'],
)


In [6]:
system_prompt = """
You are an Audio Translator, receive audio as file from a path (for e.g./content/audio.mp3)
and you run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
    splitaudio:
        Split the audio into 1-minute segments

    speech2text:
        Convert Speech to Text

    translate:
        Translate Text from Source Language to Target Language

    text2speech:
        Convert Text to Speech

Example session:

Question: Translate the audio at filepath /content/audio.mp3 from English to Malayalam.
Thought: By passing the filepath /content/audio.mp3, I need to split the audio into 1-minute segments.
Action: splitaudio
PAUSE

You will be called again with this:

Observation: I have a 1-minute audio file at filepath /content/audio.mp3

Thought: I need to convert the speech in the audio at filepath /content/audio.mp3 to english text. Pass the translated text.
Action: speech2text
PAUSE

You will be called again with this:

Observation: English Speech is converted to English text.

Thought: Need to receive the text as I need to translate the text from English to Malayalam.
Action: translate: English to Malayalam
PAUSE

You will be called again with this:

Observation: English Text translated to Malayalam.

Thought: I need to convert the translated malayalam text back to speech by passing the text.
Action: text2speech
PAUSE

You will be called again with this:

Observation: Malayalam Text is converted to speech in Malayalam.

If you have the answer, output it as the Answer.

Answer: The tranlsated audio is at /content/audio-mal.mp3 in Malayalam.

Now it's your turn:
""".strip()

In [7]:
assistant = client.beta.assistants.create(
    name="Audio Translator",
    instructions=system_prompt,
    model="gpt-4o",
  tools=[
      {
        "type": "function",
        "function":{
          "name": "splitaudio",
          "description": "Split an audio file to keep only the first 10 minutes",
          "parameters": {
            "type": "object",
            "properties": {
              "filepath": {
                "type": "string",
                "description": "The path to the audio file to be split"
              }
            },
            "required": [
              "filepath"
            ]
          }
        }
      }
      ,
      {
        "type": "function",
        "function":{
          "name": "speech2text",
          "description": "Convert speech from an audio file to text",
          "parameters": {
            "type": "object",
            "properties": {
              "filepath": {
                "type": "string",
                "description": "The path to the audio file to be transcribed"
              }
            },
            "required": [
              "filepath"
            ]
          }
        }
        }
      ,
        {
        "type": "function",
        "function":{
          "name": "translate",
          "description": "Translate text from English to a specified target language",
          "parameters": {
            "type": "object",
            "properties": {
              "text": {
                "type": "string",
                "description": "The English text to be translated"
              },
              "target_language": {
                "type": "string",
                "description": "The target language for translation"
              }
            },
            "required": [
              "text",
              "target_language"
            ]
          }
        }
        }
      ,
        {
        "type": "function",
        "function":{
          "name": "text2speech",
          "description": "Convert text to speech and save it to an audio file",
          "parameters": {
            "type": "object",
            "properties": {
              "filepath": {
                "type": "string",
                "description": "The path where the generated speech audio file will be saved"
              },
              "text": {
                "type": "string",
                "description": "The text that has be convered to speech"
              }
            },
            "required": [
              "filepath"
            ]
          }
        }
    }
  ]
)
assistant.id #asst_PuWLHnhu2BqWlDxmpyz2OvAF

'asst_PuWLHnhu2BqWlDxmpyz2OvAF'

In [8]:
!pip install pydub

In [9]:
from pydub import AudioSegment

def splitaudio(filepath: str) -> str:
  song = AudioSegment.from_mp3(filepath)

  # PyDub handles time in milliseconds
  ten_minutes = 10 * 60 * 1000

  first_10_minutes = song[:ten_minutes]

  first_10_minutes.export(filepath, format="mp3")
  return filepath

def speech2text(filepath: str) -> str:
  audio_file = open(filepath, "rb")

  transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    response_format="verbose_json",
    timestamp_granularities=["word"]
  )
  return transcript.text

def translate(text: str, target_language: str) -> str:
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": f"Translate the following English text to {target_language}: {text}"}
    ]
  )
  return response.choices[0].message.content

def text2speech(filepath:str, text:str ) -> str:
  response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=text
  )

  response.stream_to_file(filepath)
  return filepath

In [28]:
thread = client.beta.threads.create()
thread.id

'thread_b1of8JtQYJ7kNraNPlizALHN'

In [48]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Translate the audio at /content/audio-roberfrost-woodsarelovely.mp3 from English to Tamil"
)
message.id

'msg_F4IJH8KKt2m9TLAH5fVsy9No'

In [49]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id= 'asst_PuWLHnhu2BqWlDxmpyz2OvAF' #assistant.id,
)

In [53]:
import json

# run = client.beta.threads.runs.retrieve(
#   thread_id=thread.id,
#   run_id=run_id
# )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

# Define the list to store tool outputs
tool_outputs = []

# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  function_args=json.loads(tool.function.arguments)
  print(function_args)
  if tool.function.name == "splitaudio":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": splitaudio(function_args.get("filepath"))
    })
  elif tool.function.name == "speech2text":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": speech2text(function_args.get("filepath")) #function_args["filepathjson"].get("filepath")
    })
  elif tool.function.name == "translate":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": translate(function_args.get("text"), function_args.get("target_language"))
    })
  elif tool.function.name == "text2speech":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": text2speech(function_args.get("filepath"),function_args.get("text"))
    })

# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

requires_action
{'filepath': '/content/audio-roberfrost-woodsarelovely-ta.mp3', 'text': 'இவர் காடுகள் யார் என்று நான் நினைக்கிறேன். அவரது வீட்டு கிருங்காவில் உள்ளது. நான் இப்போது இல்லம் நிறுத்தி அவருடைய காடுகளை பனியால் நிகழ்ந்துள்ளவைகளை பார்த்து நிறுத்திவிடவில்லை. எனக்கு தனிமையான தாமரைகள் அவ்வரை என்று நினைக்கினில் என்ன தவம் இருக்கும் என்று அவன் எனக்கு எழுந்து கொள்ளுகிறான் காடுகள் மற்றும் உடல் ஏற்றுதலைஅடைந்த தீவிரத் தூரம், ஆணசின்னதமடைந்த ஆணச் சிம்பிளம் மட்டும். முடிவு ஒரு தவம் இருப்புகள் முரண்டு, முஶம் போகும் முறை ஒருமுறை.'}


<ipython-input-9-7a057d21442e>:42: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filepath)


Tool outputs submitted successfully.
SyncCursorPage[Message](data=[Message(id='msg_68SCLvyrSgkEaQWF1xZ6r21f', assistant_id='asst_PuWLHnhu2BqWlDxmpyz2OvAF', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Answer: The translated audio is at /content/audio-roberfrost-woodsarelovely-ta.mp3 in Tamil.'), type='text')], created_at=1721391615, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_yT6qSNPrx6u3gLKRgHWh2qfK', status=None, thread_id='thread_b1of8JtQYJ7kNraNPlizALHN'), Message(id='msg_uCUBt9PqvJq7vDWN7LlAr9M2', assistant_id='asst_PuWLHnhu2BqWlDxmpyz2OvAF', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Thought: To translate the audio file at /content/audio-roberfrost-woodsarelovely.mp3 from English to Tamil, I need to split the audio, convert the speech to text, translate the text from English to Tamil, and convert the translated te

{'filepath': '/content/audio-roberfrost-woodsarelovely-ta.mp3', 'text': 'இவர் காடுகள் யார் என்று நான் நினைக்கிறேன். அவரது வீட்டு கிருங்காவில் உள்ளது. நான் இப்போது இல்லம் நிறுத்தி அவருடைய காடுகளை பனியால் நிகழ்ந்துள்ளவைகளை பார்த்து நிறுத்திவிடவில்லை. எனக்கு தனிமையான தாமரைகள் அவ்வரை என்று நினைக்கினில் என்ன தவம் இருக்கும் என்று அவன் எனக்கு எழுந்து கொள்ளுகிறான் காடுகள் மற்றும் உடல் ஏற்றுதலைஅடைந்த தீவிரத் தூரம், ஆணசின்னதமடைந்த ஆணச் சிம்பிளம் மட்டும். முடிவு ஒரு தவம் இருப்புகள் முரண்டு, முஶம் போகும் முறை ஒருமுறை.'}

In [27]:
runs = client.beta.threads.runs.list(
  thread.id
)
#loop through runs orderby created
runs.data.sort(key=lambda x: x.created_at)
for run in runs.data:
  print(run.id)
  run_id=run.id
  print(run.status)
  print(run.created_at)
  print(run.assistant_id)

run_ZqtFvknUSe8pHPPKqO6sD5dK
completed
1721390031
asst_PuWLHnhu2BqWlDxmpyz2OvAF


In [18]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run_id
)

print(run.id)
print(run.status)

run_ZqtFvknUSe8pHPPKqO6sD5dK
completed


In [26]:
run_id

'run_ZqtFvknUSe8pHPPKqO6sD5dK'

{'text': "Whose woods these are, I think I know.
\nHis house is in the village, though.
\nHe will not see me stopping here\n
to watch his woods fill up with snow.

\n\nMy little horse must think it queer
\nto stop without a farmhouse near
\nbetween the woods and frozen lake,
\nthe darkest evening of the year.

\n\nHe gives his harness bells a shake
\nto ask if there is some mistake.
\nThe only other sound's the sweep
\nof easy wind and downy flake.
\n\nThe woods are lovely, dark and deep,
\nbut I have promises to keep,
\nand miles to go before I sleep,
\nand miles to go before I sleep.",
'target_language': 'ml'}

{'filepath': '/content/audio-roberfrost-woodsarelovely-ml.mp3', 'text': 'ഇവരുടെ കാടുകൾ ആരാണ്, എനിക്ക് തോന്നുന്നു. അവനുടെ വീട് ഗ്രാമത്തിൽ ഉണ്ട്, തോല്. ഞാൻ നിൽക്കുമ്പോൾ അവൻ എന്നെ കാണുന്നില്ല ഇവിടെ നിലംപെടുന്നതിന് അവൻ ഒരു തീര്\u200dച്ചയിലായിരിക്കും സ്നോയ്\u200cക്കുള്ള അവന്റെ കാടുകൾ കാണുന്നതിൽ നിൽനിന്ന്. എന്റെ ചെറിയ കുതിര അവിശ്വസ്തമായി തോന്നുകയാണ് കാടുകളും മുടിയടിച്ച ലേക്കടിയിലേക്ക് പടിഞ്ഞുകൂടാത്ത ഒരു സ്ഥലത്തിലും നിലേക്കരുത്, ഇവത്തെ വർഷത്തിന്റെ അത്യാന്ധർജ്ജീവിയിലും. അവൻ അവന്റെ ഹാർനെസ് ബെൽസ് ഒരു ശേകരം കൊടുക്ക പിഴവേകുകയാണ് കേർപ്പുള്ള ഉണ്ടോ എന്ന ചോദ്യം ചോദിക്കുന്നതിനായി. മറ്റൊന്നും ഓരോ ശബ്ദം എറഞ്ഞറിഞ്ഞ കാറ്റിനും മുള്ളുകൾക്കും ചിലംവഴറ്റ് കൊടുക്കുന്നതാണ്. കാടുകൾ സുന്ദരവും, ഇരുവരും അനന്തമായിരിക്കുന്നതുവരെ, എനിക്ക് നിശ്ചയം നിൽകേണ്ടവകാശങ്ങൾ ഉണ്ടകളയെന്നും, നിദ്രയിലേക്കു പോകണമെന്നും നിശ്ചയംവേണ്ടി മൈലുകൾ പോയിടെന്നും നിശ്ചയിച്ചിരിക്കുന്നു. നിദ്രയിലേക്കു പോകണമെന്ന് നിലേക്കുന്നതുകഴിഞ്ഞ് മൊത്തം മൈലുകൾ പോകണമെന്നും നിശ്ചയംവേണ്ടി നീക്കടമെന്നും നിശ്ചയിച്ചിരിക്കുന്നു.'}


{'filepath': '/content/audio-roberfrost-woodsarelovely-hi.mp3', 'text': 'जिसके ये जंगल हैं, मुझे लगता है मैं जानता हूँ। उसका घर गाँव में है, फिर भी। वह मुझे यहाँ रुकते हुए देखेगा नहीं कि उसके जंगल बर्फ से भर रहे हैं। मेरे छोटे घोड़े को विचित्र लगता होगा कि वह बिना किसी फार्महाउस के बीच रुक गया है, जंगल और जमी हुई झील के बीच, वर्ष के सबसे अंधेरे वक्त में। उसने अपनी हार्नेस की घंटियाँ हिलाकर पूछा कि कहीं कोई ग़लती तो नहीं हो गई। वही दूसरी आवाज भूपटली हवा और सूंदर कण है। ये जंगल सुंदर, अंधेरे और गहरे हैं, लेकिन मेरी एक वादे हैं जिन्हें करना है, और सोने से पहले मेरे पास लम्बी सफरें हैं, और सोने से पहले मेरे पास लम्बी सफरें हैं।'}

In [54]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  messages.data.sort(key=lambda x: x.created_at)
  # loop through thread_messages
  for message in messages.data:
      print(message.content[0].text.value)

else:
  print(run.status)

Translate the audio at /content/audio-roberfrost-woodsarelovely.mp3 from English to Malayalam
Thought: To translate the audio file at /content/audio-roberfrost-woodsarelovely.mp3 from English to Malayalam, I need to follow several steps. First, I will split the audio into 1-minute segments if necessary, then transcribe the speech to text, translate the text from English to Malayalam, and finally convert the translated text back into speech.

Action: multi_tool_use.parallel
Parameters: 
{
  "tool_uses": [
    {
      "recipient_name": "functions.splitaudio",
      "parameters": {
        "filepath": "/content/audio-roberfrost-woodsarelovely.mp3"
      }
    }
  ]
}
PAUSE
Answer: The translated audio is at /content/audio-roberfrost-woodsarelovely-ml.mp3 in Malayalam.
Translate the audio at /content/audio-roberfrost-woodsarelovely.mp3 from English to Hindi
Thought: To translate the audio file at /content/audio-roberfrost-woodsarelovely.mp3 from English to Hindi, I need to split the audio,

In [19]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)
for run in run_steps.data:
  print(run.id)
  print(run.status)
  print(run.step_details)


step_BEuxs5RPGveQFuTEpYZHHFM3
completed
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_8J5yXfNOLhexyU0e2O1qhd3f'), type='message_creation')


In [23]:
#print(json.dumps(run_steps.data, indent=2))
run_steps

SyncCursorPage[RunStep](data=[RunStep(id='step_BEuxs5RPGveQFuTEpYZHHFM3', assistant_id='asst_PuWLHnhu2BqWlDxmpyz2OvAF', cancelled_at=None, completed_at=1721390034, created_at=1721390032, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_ZqtFvknUSe8pHPPKqO6sD5dK', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_8J5yXfNOLhexyU0e2O1qhd3f'), type='message_creation'), thread_id='thread_z5UogfbJ0TMgmOoF7genVdpq', type='message_creation', usage=Usage(completion_tokens=209, prompt_tokens=835, total_tokens=1044), expires_at=None)], object='list', first_id='step_BEuxs5RPGveQFuTEpYZHHFM3', last_id='step_BEuxs5RPGveQFuTEpYZHHFM3', has_more=False)

In [ ]:
run = client.beta.threads.runs.cancel(
  thread_id=thread.id,
  run_id=run.id
)

print(run)

In [ ]:
text2speech('/content/audio-roberfrost-woodsarelovely-tn.mp3',)

In [56]:
# prompt: play an audio file

from IPython.display import Audio
Audio("/content/audio-roberfrost-woodsarelovely-ta.mp3") # Assuming the audio file is in the current directory
